# **Navigation/Mount/Env**

In [1]:
# Verify GPU Integration
%tensorflow_version 2.x 
%pip install -U --pre efficientnet
%pip install -U image-classifiers==0.2.2
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('Alert: GPU Device Not Found!')

TensorFlow 2.x selected.
     |████████████████████████████████| 81kB 2.7MB/s 


In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# Project Folder
%cd "gdrive/My Drive/projects/dermo/scripts/"

/content/gdrive/My Drive/projects/dermo/scripts


In [4]:
# Current Directory
%ls

color-io.ipynb       ensemble-val.ipynb  train-val.ipynb
ensemble-test.ipynb  __pycache__/


In [ ]:
# Libraries
from __future__ import unicode_literals
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import
import glob
import numpy as np
import cv2
from skimage import filters as skifilters
from scipy import ndimage
from skimage import filters
import matplotlib.pyplot as plt
import tqdm
from sklearn.utils import shuffle
import pandas as pd
import tensorflow as tf
import os
import h5py
import time
import json
import warnings
import dill
from sklearn.metrics import (roc_curve, auc, accuracy_score, f1_score, precision_score, 
                             recall_score, classification_report, confusion_matrix)
from scipy import interp
import efficientnet.tfkeras as efn
from classification_models.tfkeras import Classifiers as aux_models

# **I/O**

## **● Load NPY Test Dictionary**

In [ ]:
# Testing Dataset Parameters
TEST_DIR    = '../data/grayworld/test/'    
TEST_MODE   = 'val'                      # 'val'/'test'

def findScan(data, name, key):
  for i, dic in data.items():
    if dic[key] == name:        
      return i
    return -1

def test_io(path,mode):
  """"
  Input:  path
  Output: data[p] = {
      'id':
      'image':
      'label': }
  """
  if (mode=='test'):
    # Importing Images
    target_dir  = glob.glob(path+"/*.png")
    print("Number of Test Images:", len(target_dir))
    
    # Creating Dictionary
    data = {}
    for p in range(len(target_dir)):
        scan_id  =  target_dir[p].replace(".png", "")
        scan_id  =  scan_id.replace(path, "")
        # Creating List of Dictionary                    
        data[p] = {'id'    : scan_id,
                   'image' : target_dir[p] }
  
  elif (mode=='val'):
    # Importing Images
    les_dir = glob.glob(path+"/les/*.png")
    nv_dir  = glob.glob(path+"/nv/*.png")
    print("Number of LES Images:", len(les_dir))
    print("Number of NV Images:",  len(nv_dir))
    
    # Creating Dictionary (LES Scans)
    data = {}
    for p in range(len(les_dir)):
      scan_id  =  les_dir[p].replace(".png", "")
      scan_id  =  scan_id.replace(path+"/les\\", "")
      # Creating List of Dictionary                    
      data[p] = { 'id'    : scan_id,
                  'image' : les_dir[p],
                  'label' : 1 }                         # Label of LES = 1

    # Creating Dictionary (NV Scans)
    for p in range(len(nv_dir)):
      scan_id  =  nv_dir[p].replace(".png", "")
      scan_id  =  scan_id.replace(path+"/nv\\", "")
      # Creating List of Dictionary                    
      data[p+len(les_dir)] = { 'id'    : scan_id,
                               'image' : nv_dir[p],
                               'label' : 0 }            # Label of NV = 0
  return data

# Load Testing Dataset for Full Inference
test_dataset   = test_io(path=TEST_DIR, mode='test')

Number of Test Images: 1015


# **Inference**

In [ ]:
# Load Pre-Trained Models
E01          = tf.keras.models.load_model('../fin/E01.h5')         # EfficientNet-B6
# E01          = tf.keras.models.load_model('../fin/E02.h5')         # Inception-V3
# E01          = tf.keras.models.load_model('../fin/E03.h5')         # SENet-154
# E01          = tf.keras.models.load_model('../fin/E04.h5')         # SEResNeXt-101
# E01          = tf.keras.models.load_model('../fin/E05.h5')         # DenseNet-169

# Inference Parameters
TTA          = True                                                # Test-Time Augmentation
TTA_MODE     ='mean'                                               # 'mean'/'maxconf'
IO_X         = 450                                                 # Original Dimensions
IO_Y         = 600                                                 # Original Dimensions
DIM_X        = 224                                                 # Cropped Training Dimensions
DIM_Y        = 224                                                 # Cropped Training Dimensions

In [ ]:
# Populate Member Predictions on Testing Dataset
E01_predictions   = np.zeros((1,1))
all_scan_ids      = np.zeros((1,1))

for index in tqdm.tqdm(range(0,len(test_dataset))):
  # Load Data
  image           = plt.imread(test_dataset[index]['image'])
  scan_id         = test_dataset[index]['id']

  # Crop/Resize to Training Dimensions as Tensors
  image_tensor    = tf.constant(image, tf.float32)
  image_tensor    = tf.image.crop_to_bounding_box(image_tensor,(IO_X-DIM_X)//2,                        
                                                               (IO_Y-DIM_Y)//2,
                                                                DIM_X,DIM_Y) 
  if (TTA==True):
    # Test-Time Augmentation
    image01_tensor  = tf.clip_by_value(tf.image.flip_left_right(image_tensor),0.0,1.0)                            # TTA 01: Horizontal Flip
    image02_tensor  = tf.clip_by_value(tf.image.flip_up_down(image_tensor),0.0,1.0)                               # TTA 02: Vertical Flip
    image03_tensor  = tf.clip_by_value(tf.image.adjust_brightness(image_tensor,delta=16.0/255.0),0.0,1.0)         # TTA 03: Brightness Shift
    image04_tensor  = tf.clip_by_value(tf.image.adjust_saturation(image_tensor,saturation_factor=1.5),0.0,1.0)    # TTA 04: Saturation Shift
    image05_tensor  = tf.clip_by_value(tf.image.adjust_contrast(image_tensor,contrast_factor=1.5),0.0,1.0)        # TTA 05: Contrast Boost
    # All Predictions
    all_predictions = np.array((E01.predict(np.expand_dims(image_tensor.numpy(),axis=0)),
                                E01.predict(np.expand_dims(image01_tensor.numpy(),axis=0)),
                                E01.predict(np.expand_dims(image02_tensor.numpy(),axis=0)),
                                E01.predict(np.expand_dims(image03_tensor.numpy(),axis=0)),
                                E01.predict(np.expand_dims(image04_tensor.numpy(),axis=0)),
                                E01.predict(np.expand_dims(image05_tensor.numpy(),axis=0))))
    if (TTA_MODE=='mean'):
      # Average Predictions
      TTA_prediction  = np.average(all_predictions)
    elif (TTA_MODE=='maxconf'):
      # Max Confidence Prediction
      maxconf_arg     = np.argmax((abs(0.5-all_predictions)))
      TTA_prediction  = (all_predictions[maxconf_arg]).mean()
    
  elif (TTA==False):
    TTA_prediction  = model.predict(np.expand_dims(image_tensor.numpy(),axis=0)) 
  
  # Aggregate Predictions
  E01_predictions   = np.concatenate((E01_predictions,TTA_prediction.reshape(-1,1)), axis=0)
  all_scan_ids      = np.concatenate((all_scan_ids,np.array(scan_id).reshape(-1,1)), axis=0)

E01_predictions = E01_predictions[1:]
all_scan_ids    = all_scan_ids[1:]

100%|██████████| 1015/1015 [11:07<00:00,  1.53it/s]


In [ ]:
# Save Predictions
predictions    =  pd.DataFrame(list(zip(all_scan_ids, E01_predictions)),
columns        =  ['scan_id','prob'])
predictions.to_csv('../test_pred/E01_testset.csv', encoding='utf-8', index=False)